# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Read and clean up the csv
cities = pd.read_csv('../output_data/cities_final.csv')
cities.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kapaa,22.08,-159.32,19.44,59,6,4.58,US,1608222349
1,1,Kenai,60.55,-151.26,-5.56,85,1,5.10,US,1608222350
2,2,Province of Mantua,45.17,10.78,8.33,87,75,0.50,IT,1608222351
3,3,Petropavlovsk-Kamchatskiy,53.05,158.65,-11.00,78,90,4.00,RU,1608222352
4,4,Araouane,18.90,-3.53,30.46,13,65,2.53,ML,1608222354


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations and convert Lat, Lng, Humidity to float 
city_locations = cities[["Lat", "Lng"]].astype(float)
city_humidity = cities["Humidity"].astype(float)

#Create heatmap layer
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(figure_layout)
heat_layer = gmaps.heatmap_layer(city_locations, weights=city_humidity, dissipating=False, max_intensity=100, point_radius = 2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#Narrow down the DataFrame to ideal weather conditions
ideal_weather = cities.loc[(cities["Max Temp"] >= 25) & (cities["Max Temp"] <= 30) & (cities["Cloudiness"]==0) & 
                           (cities["Wind Speed"] < 10)]
ideal_weather = ideal_weather.dropna(how='any')
ideal_weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
56,56,Rawson,-43.30,-65.10,25.57,24,0,8.51,AR,1608222432
98,98,Illapel,-31.63,-71.17,28.14,27,0,4.60,CL,1608222500
114,114,Nouadhibou,20.93,-17.03,27.00,34,0,9.80,MR,1608222470
157,157,Puerto Ayora,-0.74,-90.35,25.00,69,0,7.70,EC,1608222602
347,347,Şabyā,17.15,42.63,28.00,69,0,2.10,SA,1608222904


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = ideal_weather
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
56,56,Rawson,-43.30,-65.10,25.57,24,0,8.51,AR,1608222432,
98,98,Illapel,-31.63,-71.17,28.14,27,0,4.60,CL,1608222500,
114,114,Nouadhibou,20.93,-17.03,27.00,34,0,9.80,MR,1608222470,
157,157,Puerto Ayora,-0.74,-90.35,25.00,69,0,7.70,EC,1608222602,
347,347,Şabyā,17.15,42.63,28.00,69,0,2.10,SA,1608222904,


In [11]:
# List to store requested hotel name
hotels = []
#Google nearby search url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# define parameters
params = { 
    "radius": 5000, 
    "type": "lodging",
    "key": g_key
}
# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotel_df.iterrows():
    
    latit = row["Lat"]
    long = row["Lng"]
    
    # update location key value
    params["location"] = f"{latit}, {long}"
    
    
    response = requests.get(base_url, params=params).json()
    #print(json.dumps(response, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        #hotels.append(response["results"][0]["name"])
    except (IndexError):
        print("Missing field... skipping.")


In [13]:
#Check populated DF
hotel_df
#remove blank/null values
#hotel_df = hotel_df.dropna(how='any', axis=0)
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
56,56,Rawson,-43.30,-65.10,25.57,24,0,8.51,AR,1608222432,Hotel Punta León
98,98,Illapel,-31.63,-71.17,28.14,27,0,4.60,CL,1608222500,Hotel Londres
114,114,Nouadhibou,20.93,-17.03,27.00,34,0,9.80,MR,1608222470,El Medina
157,157,Puerto Ayora,-0.74,-90.35,25.00,69,0,7.70,EC,1608222602,Finch Bay Galapagos Hotel
347,347,Şabyā,17.15,42.63,28.00,69,0,2.10,SA,1608222904,Aseel Hotel Apartment
369,369,Jerez,14.10,-89.75,28.00,54,0,2.10,GT,1608222934,PRIMER IGLESIA BAUTISTA MONTE LOS OLIVOS EL TA...
467,467,Caravelas,-17.71,-39.25,29.24,54,0,4.38,BR,1608223076,Pousada dos Navegantes
468,468,Acajutla,13.59,-89.83,28.00,54,0,2.10,SV,1608223077,Urbanización Brisas de Acajutla II
471,471,Meïganga,6.52,14.30,26.12,18,0,2.78,CM,1608223081,DFR HOUSE
550,550,Atar,20.52,-13.05,27.00,28,0,7.20,MR,1608223192,Oumou Elghoura


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(heat_layer) 
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))